In [1]:
import pandas as pd
import numpy as np
import torch
import os
from tqdm import tqdm
from torchtext import data

In [ ]:
!pwd

/content


In [2]:
data_dir = 'drive/My Drive/tx/'

# texts = pd.read_pickle(os.path.join(data_dir, 'user_creative_ids.pkl'))
labels = pd.read_pickle(os.path.join(data_dir, 'user_gender_cls2.pkl'))

In [3]:
labels.value_counts()

0    602610
1    297390
Name: gender, dtype: int64

In [ ]:
# df = pd.DataFrame(texts)

In [ ]:
# df['gender'] = labels

In [ ]:
# df.iloc[:800000].to_pickle(os.path.join(data_dir, 'train_gender.pkl'))
# df.iloc[800000:].to_pickle(os.path.join(data_dir, 'val_gender.pkl'))

In [ ]:
df.head(2
        )

,creative_id,gender
user_id,,
1,"[877468, 209778, 821396, 1683713, 122032, 7169...",0
10,"[134938, 69204, 197464, 81006, 3331781, 71689,...",1


In [ ]:
texts.head(2)

user_id
1     [877468, 209778, 821396, 1683713, 122032, 7169...
10    [134938, 69204, 197464, 81006, 3331781, 71689,...
Name: creative_id, dtype: object

In [ ]:
texts.shape

(900000,)

In [ ]:
labels.head(2)

user_id
1    0
2    0
Name: gender, dtype: int64

In [ ]:
labels.shape

(900000,)

In [7]:
# TEXT = data.Field(sequential=True, fix_length=50, batch_first=True)
# LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)

TEXT = data.Field(sequential=True, fix_length=50)
LABEL = data.Field(sequential=False, use_vocab=False)

In [4]:
# 定义Dataset
class MyDataset(data.Dataset):
    name = 'TX Dataset'

    @staticmethod
    def sort_key(ex):
        return len(ex.creative_ids)

    def __init__(self, path, text_field, label_field, test=False, aug=False, **kwargs):
        fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("creative_ids", text_field), ("gender", label_field)]
        
        examples = []
        df = pd.read_pickle(path)
        print('read data from {}'.format(path))

        if test:
            # 如果为测试集，则不加载label
            for text in tqdm(csv_data['comment_text']):
                examples.append(data.Example.fromlist([None, text, None], fields))
        else:
            for text, label in tqdm(zip(df['creative_id'], df['gender'])):
                if aug:
                    # do augmentation
                    rate = random.random()
                    if rate > 0.5:
                        text = self.dropout(text)
                    else:
                        text = self.shuffle(text)
                # Example: Defines a single training or test example.Stores each column of the example as an attribute.
                examples.append(data.Example.fromlist([None, text, label], fields))
        # 之前是一些预处理操作，此处调用super调用父类构造方法，产生标准Dataset
        # super(MyDataset, self).__init__(examples, fields, **kwargs)
        super(MyDataset, self).__init__(examples, fields)

    def shuffle(self, text):
        text = np.random.permutation(text.strip().split())
        return ' '.join(text)

    def dropout(self, text, p=0.5):
        # random delete some text
        text = text.strip().split()
        len_ = len(text)
        indexs = np.random.choice(len_, int(len_ * p))
        for i in indexs:
            text[i] = ''
        return ' '.join(text)


In [8]:
train = MyDataset(os.path.join(data_dir, 'train_gender.pkl'), text_field=TEXT, label_field=LABEL, test=False, aug=False)

33321it [00:00, 333202.37it/s]

read data from drive/My Drive/tx/train_gender.pkl


800000it [00:04, 164624.10it/s]


In [9]:
val = MyDataset(os.path.join(data_dir, 'val_gender.pkl'), text_field=TEXT, label_field=LABEL, test=False, aug=False)

30663it [00:00, 306627.37it/s]

read data from drive/My Drive/tx/val_gender.pkl


100000it [00:01, 93339.42it/s]


In [ ]:
train[0].__dict__.keys()

dict_keys(['creative_ids', 'gender'])

In [ ]:
train[0].creative_ids

['877468',
 '209778',
 '821396',
 '1683713',
 '122032',
 '71691',
 '1940159',
 '90171',
 '2087846',
 '2456345',
 '3068256',
 '2456345',
 '3592928']

In [ ]:
train[0].gender

0

In [10]:
TEXT.build_vocab(train)

In [ ]:
TEXT.vocab.freqs.most_common(10)

[('108133', 103033),
 ('37298', 84043),
 ('304118', 73876),
 ('1837000', 70875),
 ('32571', 69956),
 ('12838', 69813),
 ('54150', 59148),
 ('1779084', 50391),
 ('2353377', 46829),
 ('47512', 44533)]

In [ ]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             '108133': 2,
             '37298': 3,
             '304118': 4,
             '1837000': 5,
             '32571': 6,
             '12838': 7,
             '54150': 8,
             '1779084': 9,
             '2353377': 10,
             '47512': 11,
             '37295': 12,
             '670': 13,
             '24333': 14,
             '1807219': 15,
             '2085566': 16,
             '2145432': 17,
             '7400': 18,
             '69204': 19,
             '39714': 20,
             '33944': 21,
             '1836990': 22,
             '649132': 23,
             '599224': 24,
             '11050': 25,
             '311867': 26,
             '1662565': 27,
             '648402': 28,
             '599128': 29,
             '1836995': 30,
             '114074': 31,
             '108136': 32,
             '441518': 33,
             '2361420': 34,
             '

In [11]:
from torchtext.data import Iterator, BucketIterator

In [12]:
train_iter, val_iter = BucketIterator.splits(
        (train, val), # 构建数据集所需的数据集
        batch_sizes=(100, 100),
#         device=-1, # 如果使用gpu，此处将-1更换为GPU的编号
        # device='cpu',
        # sort_key=lambda x: len(x.creative_ids), # the BucketIterator needs to be told what function it should use to group the data.
        # sort_within_batch=False,
        # repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

In [ ]:
for idx, batch in enumerate(train_iter):
    print(batch)
    text, label = batch.creative_ids, batch.gender
    # print(text.shape, label.shape)
    print(text)
    print(label)
    break


[torchtext.data.batch.Batch of size 8 from TX DATASET]
	[.creative_ids]:[torch.LongTensor of size 50x8]
	[.gender]:[torch.LongTensor of size 8]
tensor([[     74, 1230152,  232971,  119402,    3434,    7113,   27829,    1140],
        [ 167650,   37682,   21197,    9489,     900,   88870,   10564,   32110],
        [   4721,  166163,   51999,   14703,    6522,      36,  143622,      23],
        [   2162,      61,  177690,    6507,   19883,   11508,    6741,      56],
        [1411804,  164988,    3251,   42776,   11529,  138281,   24096,   37313],
        [  17203,    1120,   79620,  234563,  263149,  318187,    8596,   14304],
        [ 302202,  192697,   81400,    1159,    7377,   16412,   72286,      56],
        [   6245,      40,  176312,   14306,       4,  110281,    3669,    8854],
        [ 765443,    2337,    6817,  566659,      49,   13428,   26449,    1211],
        [   7486,   31074,   11357,  931844,      49,  110281,  541980,   11407],
        [1069806,   32807,     635,

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
import math

In [14]:
class LearnedPositionEncoding(nn.Embedding):
    def __init__(self,d_model, dropout = 0.1,max_len = 5000):
        super().__init__(max_len, d_model)
        self.dropout = nn.Dropout(p = dropout)
     
    def forward(self, x):
        weight = self.weight.data.unsqueeze(1)
        x = x + weight[:x.size(0),:]
        return self.dropout(x)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model,dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)        

In [15]:
class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()
        self.word_embeddings = nn.Embedding(len(TEXT.vocab), 100)  # embedding之后的shape: torch.Size([200, 8, 300])
        # 若使用预训练的词向量，需在此处指定预训练的权重
        # embedding.weight.data.copy_(weight_matrix)
        # self.lstm = nn.LSTM(input_size=100, hidden_size=128, num_layers=1, bidirectional=True)  # torch.Size([200, 8, 128])
        self.pos_encoding = LearnedPositionEncoding(100)
        # self.pos_encoding = PositionalEncoding(100)
        encoder_layer = nn.TransformerEncoderLayer(d_model=100, nhead=2)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)  
        self.decoder = nn.Linear(100, 2)

    def forward(self, sentence, src_padding_mask=None):
        embeds = self.word_embeddings(sentence)
        embeds = self.pos_encoding(embeds)
        # lstm_out = self.lstm(embeds)[0]  # lstm_out:200x8x128
        trans = self.transformer_encoder(embeds, src_key_padding_mask=src_padding_mask)
        # print(lstm_out.shape)
        # 取最后一个时间步
        # final = trans[-1]  # 8*128
        # final, _ = trans.max(0)
        final = trans.mean(0)
        # print(final.shape)
        y = self.decoder(final)  # 8*2 
        return y

In [65]:
model = LSTM()
model.to('cuda')
model.train()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
# criterion = nn.CrossEntropyLoss()

criterion = MyLoss

In [21]:
preds = torch.tensor([[-0.0450, -0.0435],
        [ 0.0063,  0.0288],
        [ 0.2572, -0.1292],
        [ 0.1535, -0.1328],
        [ 0.2729, -0.2122],
        [ 0.1859, -0.1356],
        [ 0.1851,  0.0081],])
labels = torch.tensor([0, 0, 1, 1, 0, 1, 0,])

In [22]:
preds = F.softmax(preds, -1)

In [23]:
preds

tensor([[0.4996, 0.5004],
        [0.4944, 0.5056],
        [0.5954, 0.4046],
        [0.5711, 0.4289],
        [0.6190, 0.3810],
        [0.5797, 0.4203],
        [0.5441, 0.4559]])

In [26]:
labels.unsqueeze(1)

tensor([[0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0]])

In [29]:
t = torch.gather(preds, 1, labels.unsqueeze(1))
t

tensor([[0.4996],
        [0.4944],
        [0.4046],
        [0.4289],
        [0.6190],
        [0.4203],
        [0.5441]])

In [51]:
loss = -(1-t)*torch.log(t)
loss

tensor([[0.3472],
        [0.3562],
        [0.5388],
        [0.4834],
        [0.1828],
        [0.5025],
        [0.2774]])

In [48]:
class_weights = [0.3, 0.7]
class_weights = torch.tensor(class_weights)

In [49]:
class_weights.gather(0, labels)

tensor([0.3000, 0.3000, 0.7000, 0.7000, 0.3000, 0.7000, 0.3000])

In [54]:
loss = loss.squeeze() * class_weights.gather(0, labels)

In [55]:
loss

tensor([0.1042, 0.1069, 0.3772, 0.3384, 0.0548, 0.3517, 0.0832])

In [57]:
loss.mean()

tensor(0.2023)

In [56]:
loss.mean(0)

tensor(0.2023)

In [64]:
def MyLoss(preds, labels, class_weights=[1, 1.2], gamma=1):
    class_weights = torch.tensor(class_weights).to('cuda')
    class_weights = class_weights.gather(0, labels)
    preds = F.softmax(preds, -1)
    t = torch.gather(preds, 1, labels.unsqueeze(1))
    loss = -(1-t)**gamma * torch.log(t)
    loss = loss.squeeze() * class_weights
    return loss.mean()

In [ ]:
len(train_iter)

8000

In [ ]:
tensor([[ 0.0163, -0.1587],
        [ 0.1188, -0.2245],
        [ 0.0598, -0.1625],
        [-0.2124,  0.1388],
        [-0.1100,  0.0684],
        [-0.0453, -0.1331],
        [ 0.0188, -0.2474],
        [ 0.0050, -0.1964]], device='cuda:0')
tensor([0, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')

In [66]:
time_start = datetime.now()
for step, batch in enumerate(train_iter):
    
    optimizer.zero_grad()
    textsb = batch.creative_ids.to('cuda')
    textsb_mask = (textsb==1).transpose(0, 1)
    labelsb = batch.gender.to('cuda')
    predicted = model(textsb, textsb_mask)
    # print(predicted)
    # print(labelsb)
    # break
    loss = criterion(predicted, labelsb)
    loss.backward()
    optimizer.step()
    if (step+1) % 1000 == 0 or step==0:
        # print(loss)
        correct_val = 0.
        total_val = 0.
        loss_val = 0.
        model.eval()
        with torch.no_grad():
            for stepv, batchv in enumerate(val_iter):
              textsb = batchv.creative_ids.to('cuda')
              textsb_mask = (textsb==1).transpose(0, 1)
              labelsb = batchv.gender.to('cuda')
              outputs = model(textsb, textsb_mask)
              # print(outputs)
              loss_val += criterion(outputs, labelsb).item()
              _, predicted = torch.max(outputs, 1)
              # print(predicted)
              total_val += labelsb.size(0)
              correct_val += (predicted == labelsb).squeeze().sum().to('cpu').numpy()
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%d %H:%M:%S")} {int((now-time_start).total_seconds())}s step: {step}; train loss: {loss.item()}; val loss: {loss_val/(stepv+1)}; accuracy: {correct_val/total_val}')
        model.train()
        time_start = now

2020-07-30 10:10:49 5s step: 0; train loss: 0.37468740344047546; val loss: 2.4653921900987625; accuracy: 0.67141
2020-07-30 10:12:40 111s step: 999; train loss: 0.23959611356258392; val loss: 0.181330590672791; accuracy: 0.86487
2020-07-30 10:14:31 111s step: 1999; train loss: 0.20852698385715485; val loss: 0.16422740852087736; accuracy: 0.88147
2020-07-30 10:16:23 111s step: 2999; train loss: 0.18095630407333374; val loss: 0.15023894307389857; accuracy: 0.89608
2020-07-30 10:18:14 111s step: 3999; train loss: 0.13235890865325928; val loss: 0.14419522135332227; accuracy: 0.90023
2020-07-30 10:20:05 111s step: 4999; train loss: 0.17433345317840576; val loss: 0.1391603584587574; accuracy: 0.90222
2020-07-30 10:21:56 111s step: 5999; train loss: 0.11843869835138321; val loss: 0.1356759348474443; accuracy: 0.90472
2020-07-30 10:23:47 111s step: 6999; train loss: 0.16894489526748657; val loss: 0.13879442168399692; accuracy: 0.90748
2020-07-30 10:25:39 111s step: 7999; train loss: 0.12020215

In [ ]:
# transformer heads 20 + learnable pos_encode
2020-07-29 10:23:30 11s step: 0; train loss: 0.6888568997383118; val loss: 2.459111626148224; accuracy: 0.67141
2020-07-29 10:26:16 166s step: 999; train loss: 0.24504294991493225; val loss: 0.27081938640028236; accuracy: 0.89253
2020-07-29 10:29:03 167s step: 1999; train loss: 0.2326202541589737; val loss: 0.24225042283535003; accuracy: 0.90461
2020-07-29 10:31:51 168s step: 2999; train loss: 0.2733555734157562; val loss: 0.23457658088207245; accuracy: 0.90829
2020-07-29 10:34:40 168s step: 3999; train loss: 0.25496596097946167; val loss: 0.22565073600411414; accuracy: 0.91211
2020-07-29 10:37:29 169s step: 4999; train loss: 0.2791408896446228; val loss: 0.22181752740591765; accuracy: 0.91498
2020-07-29 10:40:18 169s step: 5999; train loss: 0.2624041438102722; val loss: 0.21999888399615883; accuracy: 0.91686
2020-07-29 10:43:07 169s step: 6999; train loss: 0.17159701883792877; val loss: 0.21413596580177546; accuracy: 0.91737
2020-07-29 10:45:56 169s step: 7999; train loss: 0.39365866780281067; val loss: 0.2206704160310328; accuracy: 0.91921
# transformer heads 10 + learnable pos_encode
head 增大后，效果变好
2020-07-29 09:59:44 10s step: 0; train loss: 0.7531651258468628; val loss: 2.2926154747009275; accuracy: 0.67141
2020-07-29 10:02:26 162s step: 999; train loss: 0.26687565445899963; val loss: 0.28532810951024296; accuracy: 0.88035
2020-07-29 10:05:09 163s step: 1999; train loss: 0.19847282767295837; val loss: 0.2500127937942743; accuracy: 0.90012
2020-07-29 10:07:54 164s step: 2999; train loss: 0.2927064895629883; val loss: 0.2390718938037753; accuracy: 0.9071
2020-07-29 10:10:39 165s step: 3999; train loss: 0.23669065535068512; val loss: 0.23182151336222886; accuracy: 0.90987
2020-07-29 10:13:24 165s step: 4999; train loss: 0.2145664542913437; val loss: 0.2255424172580242; accuracy: 0.91288
2020-07-29 10:16:10 165s step: 5999; train loss: 0.22272655367851257; val loss: 0.22508002539351582; accuracy: 0.91444
2020-07-29 10:18:56 166s step: 6999; train loss: 0.16925068199634552; val loss: 0.21928808401152491; accuracy: 0.9156
2020-07-29 10:21:42 165s step: 7999; train loss: 0.12449522316455841; val loss: 0.2184925230629742; accuracy: 0.91716
# transformer heads 4 + learnable pos_encode
2020-07-29 09:11:30 161s step: 999; train loss: 0.3133758008480072; val loss: 0.3025586664676666; accuracy: 0.8772
2020-07-29 09:14:11 161s step: 1999; train loss: 0.45509541034698486; val loss: 0.27888034814596174; accuracy: 0.88685
2020-07-29 09:16:53 162s step: 2999; train loss: 0.2985176146030426; val loss: 0.24819827196747066; accuracy: 0.90179
2020-07-29 09:19:36 162s step: 3999; train loss: 0.2637624740600586; val loss: 0.2388577264174819; accuracy: 0.90579
2020-07-29 09:22:19 163s step: 4999; train loss: 0.316198468208313; val loss: 0.23650204017013313; accuracy: 0.90699
2020-07-29 09:25:03 163s step: 5999; train loss: 0.1848619431257248; val loss: 0.2301157729178667; accuracy: 0.91115
2020-07-29 09:27:47 163s step: 6999; train loss: 0.25114160776138306; val loss: 0.22648274568468332; accuracy: 0.91385
2020-07-29 09:30:31 163s step: 7999; train loss: 0.1501266360282898; val loss: 0.22862565544620156; accuracy: 0.91331
# transformer heads 2 + learnable pos_encode + padding mask + encode layer 2(其它的layer为1)
大跌眼镜啊，可能因为参数多，收敛起来慢吧
2020-07-29 11:22:06 15s step: 0; train loss: 0.7336103320121765; val loss: 2.5716112222671508; accuracy: 0.67141
2020-07-29 11:25:09 183s step: 999; train loss: 0.5104204416275024; val loss: 0.5579511325657368; accuracy: 0.67559
2020-07-29 11:28:14 184s step: 1999; train loss: 0.5596928596496582; val loss: 0.5105156623125077; accuracy: 0.74181
2020-07-29 11:31:19 185s step: 2999; train loss: 0.4652661085128784; val loss: 0.4614643865823746; accuracy: 0.78875
2020-07-29 11:34:25 186s step: 3999; train loss: 0.4260783791542053; val loss: 0.44898554304242133; accuracy: 0.79963
2020-07-29 11:37:32 186s step: 4999; train loss: 0.3804198205471039; val loss: 0.4218429928719997; accuracy: 0.80995
2020-07-29 11:40:39 186s step: 5999; train loss: 0.4024531841278076; val loss: 0.4213450171351433; accuracy: 0.81189
2020-07-29 11:43:46 186s step: 6999; train loss: 0.47317180037498474; val loss: 0.39196812753379345; accuracy: 0.82132
2020-07-29 11:46:53 187s step: 7999; train loss: 0.33840930461883545; val loss: 0.35882561206817626; accuracy: 0.84294

# transformer heads 2 + learnable pos_encode + padding mask + 修改loss focal loss(alpha=1, gamaa=2)
2020-07-30 09:32:33 5s step: 0; train loss: 0.19623416662216187; val loss: 1.7654534958600998; accuracy: 0.67141
2020-07-30 09:34:23 110s step: 999; train loss: 0.08362846076488495; val loss: 0.08525678870454431; accuracy: 0.86156
2020-07-30 09:36:13 110s step: 1999; train loss: 0.08869556337594986; val loss: 0.07494684437662363; accuracy: 0.88521
2020-07-30 09:38:03 110s step: 2999; train loss: 0.08733316510915756; val loss: 0.07229035603068769; accuracy: 0.89652
2020-07-30 09:39:53 110s step: 3999; train loss: 0.06403527408838272; val loss: 0.0680099964775145; accuracy: 0.89852
2020-07-30 09:41:43 110s step: 4999; train loss: 0.04991994798183441; val loss: 0.06649616079498083; accuracy: 0.90771
2020-07-30 09:43:33 110s step: 5999; train loss: 0.05683526769280434; val loss: 0.06281682370975614; accuracy: 0.9089
2020-07-30 09:45:23 110s step: 6999; train loss: 0.06996878236532211; val loss: 0.06265883174072952; accuracy: 0.91068
2020-07-30 09:47:13 110s step: 7999; train loss: 0.07312659174203873; val loss: 0.06251069465652108; accuracy: 0.91157

# transformer heads 2 + learnable pos_encode + padding mask + 修改loss focal loss(class_weights=[1, 2], gamaa=1)
0    602610
1    297390
按照经验，这点不均衡根本不算什么，class_weights=[1, 2]过头了， 果不其然！
2020-07-30 09:53:49 6s step: 0; train loss: 0.4894126057624817; val loss: 3.892231452465057; accuracy: 0.67141
2020-07-30 09:55:40 111s step: 999; train loss: 0.2477148324251175; val loss: 0.23363508321344853; accuracy: 0.84886
2020-07-30 09:57:31 111s step: 1999; train loss: 0.1983683854341507; val loss: 0.20802405197173357; accuracy: 0.88532
2020-07-30 09:59:22 111s step: 2999; train loss: 0.1671139895915985; val loss: 0.19373685778304933; accuracy: 0.89232
2020-07-30 10:01:13 111s step: 3999; train loss: 0.21354083716869354; val loss: 0.18876031852141023; accuracy: 0.89108
2020-07-30 10:03:05 111s step: 4999; train loss: 0.16969595849514008; val loss: 0.1800823621377349; accuracy: 0.89913
2020-07-30 10:04:56 111s step: 5999; train loss: 0.36961841583251953; val loss: 0.1766268918439746; accuracy: 0.89844
2020-07-30 10:06:47 111s step: 6999; train loss: 0.17424160242080688; val loss: 0.17147966679185628; accuracy: 0.89647
2020-07-30 10:08:38 111s step: 7999; train loss: 0.15292054414749146; val loss: 0.17202003279328346; accuracy: 0.8964

class_weights=[1, 1.2]， 效果差强人意，结论就是 这点不均衡，根本不用调class_weights...
2020-07-30 10:10:49 5s step: 0; train loss: 0.37468740344047546; val loss: 2.4653921900987625; accuracy: 0.67141
2020-07-30 10:12:40 111s step: 999; train loss: 0.23959611356258392; val loss: 0.181330590672791; accuracy: 0.86487
2020-07-30 10:14:31 111s step: 1999; train loss: 0.20852698385715485; val loss: 0.16422740852087736; accuracy: 0.88147
2020-07-30 10:16:23 111s step: 2999; train loss: 0.18095630407333374; val loss: 0.15023894307389857; accuracy: 0.89608
2020-07-30 10:18:14 111s step: 3999; train loss: 0.13235890865325928; val loss: 0.14419522135332227; accuracy: 0.90023
2020-07-30 10:20:05 111s step: 4999; train loss: 0.17433345317840576; val loss: 0.1391603584587574; accuracy: 0.90222
2020-07-30 10:21:56 111s step: 5999; train loss: 0.11843869835138321; val loss: 0.1356759348474443; accuracy: 0.90472
2020-07-30 10:23:47 111s step: 6999; train loss: 0.16894489526748657; val loss: 0.13879442168399692; accuracy: 0.90748
2020-07-30 10:25:39 111s step: 7999; train loss: 0.12020215392112732; val loss: 0.13012073561176657; accuracy: 0.91085

# transformer heads 2 + learnable pos_encode + padding mask + 修改loss focal loss(class_weights=[], gamaa=1)
好开森，用着自己实现的loss，好爽！
2020-07-30 09:12:48 5s step: 0; train loss: 0.3496469557285309; val loss: 2.5490412428379057; accuracy: 0.67141
2020-07-30 09:14:38 109s step: 999; train loss: 0.1401101052761078; val loss: 0.17717842800542713; accuracy: 0.84789
2020-07-30 09:16:28 109s step: 1999; train loss: 0.16289111971855164; val loss: 0.1498344074897468; accuracy: 0.88093
2020-07-30 09:18:17 109s step: 2999; train loss: 0.14678867161273956; val loss: 0.1439799223802984; accuracy: 0.88556
2020-07-30 09:20:07 109s step: 3999; train loss: 0.19225876033306122; val loss: 0.1287810543626547; accuracy: 0.90165
2020-07-30 09:21:57 109s step: 4999; train loss: 0.12752223014831543; val loss: 0.12818017444759608; accuracy: 0.90222
2020-07-30 09:23:47 110s step: 5999; train loss: 0.16702598333358765; val loss: 0.12510130172967912; accuracy: 0.9024
2020-07-30 09:25:37 110s step: 6999; train loss: 0.12674368917942047; val loss: 0.12303619818016887; accuracy: 0.90838
2020-07-30 09:27:27 110s step: 7999; train loss: 0.08951617777347565; val loss: 0.11706878117099405; accuracy: 0.91208


# transformer heads 2 + learnable pos_encode + padding mask
加上mask后效果提升！
2020-07-29 10:57:52 10s step: 0; train loss: 0.7252820730209351; val loss: 2.3158143815994263; accuracy: 0.67141
2020-07-29 11:00:32 159s step: 999; train loss: 0.3290337324142456; val loss: 0.32025137323886155; accuracy: 0.86478
2020-07-29 11:03:13 160s step: 1999; train loss: 0.1966371387243271; val loss: 0.28699284441024064; accuracy: 0.88216
2020-07-29 11:05:55 162s step: 2999; train loss: 0.3227078914642334; val loss: 0.2588016359135509; accuracy: 0.8967
2020-07-29 11:08:38 162s step: 3999; train loss: 0.3194979727268219; val loss: 0.25239473562687637; accuracy: 0.90147
2020-07-29 11:11:21 162s step: 4999; train loss: 0.317527174949646; val loss: 0.2572562959343195; accuracy: 0.90446
2020-07-29 11:14:04 163s step: 5999; train loss: 0.17862893640995026; val loss: 0.24698179376870394; accuracy: 0.90461
2020-07-29 11:16:47 163s step: 6999; train loss: 0.1881033331155777; val loss: 0.24531507790088652; accuracy: 0.9091
2020-07-29 11:19:30 163s step: 7999; train loss: 0.2803882658481598; val loss: 0.2318588673248887; accuracy: 0.91223

# transformer heads 2 + learnable pos_encode
2020-07-29 08:44:18 9s step: 0; train loss: 0.6819533705711365; val loss: 2.323913005232811; accuracy: 0.67141
2020-07-29 08:46:57 159s step: 1000; train loss: 0.44977420568466187; val loss: 0.35437445545196533; accuracy: 0.84789
2020-07-29 08:49:38 160s step: 2000; train loss: 0.4483421742916107; val loss: 0.2991078807041049; accuracy: 0.87619
2020-07-29 08:52:19 161s step: 3000; train loss: 0.2626582980155945; val loss: 0.2861929105371237; accuracy: 0.88346
2020-07-29 08:55:01 161s step: 4000; train loss: 0.23002101480960846; val loss: 0.2698283971697092; accuracy: 0.8959
2020-07-29 08:57:44 162s step: 5000; train loss: 0.21724306046962738; val loss: 0.2594862695336342; accuracy: 0.90002
2020-07-29 09:00:26 162s step: 6000; train loss: 0.2609153687953949; val loss: 0.24245518317818643; accuracy: 0.90455
2020-07-29 09:03:09 162s step: 7000; train loss: 0.25595805048942566; val loss: 0.23996299289539455; accuracy: 0.9063
# transformer heads 2 + unlearnable pos_encode
表现比learnable pos_encode略差一些
2020-07-29 09:35:23 9s step: 0; train loss: 0.6293551921844482; val loss: 2.017247985124588; accuracy: 0.67141
2020-07-29 09:38:03 159s step: 999; train loss: 0.2743014693260193; val loss: 0.34635482528060674; accuracy: 0.8458
2020-07-29 09:40:43 160s step: 1999; train loss: 0.2392614185810089; val loss: 0.2934611782804131; accuracy: 0.87732
2020-07-29 09:43:24 161s step: 2999; train loss: 0.3048652410507202; val loss: 0.2768153970837593; accuracy: 0.88686
2020-07-29 09:46:06 161s step: 3999; train loss: 0.2781606912612915; val loss: 0.2623164827004075; accuracy: 0.89457
2020-07-29 09:48:48 161s step: 4999; train loss: 0.24915334582328796; val loss: 0.2541348894014955; accuracy: 0.89724
2020-07-29 09:51:30 162s step: 5999; train loss: 0.24871185421943665; val loss: 0.2470167309269309; accuracy: 0.90174
2020-07-29 09:54:12 162s step: 6999; train loss: 0.24821065366268158; val loss: 0.2557852273881435; accuracy: 0.90135
# transformer heads 2 without pos_encode
2020-07-29 08:37:13 23s step: 7800; train loss: 0.22911757230758667; val loss: 0.2624150919392705; accuracy: 0.89868
2020-07-29 08:37:37 23s step: 7900; train loss: 0.1702890545129776; val loss: 0.25887332943081853; accuracy: 0.89878